In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot
from pandas import datetime

<ipython-input-1-878b61d2ea0c>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [2]:
## Read CSV file generated from clean_temps.ipynb
data = pd.read_csv("county_max_temps.csv")

In [3]:
## Drop rows.
data = data.drop(['Unnamed: 0', 'ID', 'FIPS'], axis = 1)

In [4]:
## Average monthly temperature and group by year and state.
data['Avg_Temp'] = data[['Jan','Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']].mean(axis = 1)
data = data.groupby(['Year', 'State'], as_index = False).mean()

,Year,State,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Avg_Temp
0,1895,AL,52.389552,47.768657,66.219403,75.511940,80.502985,88.464179,89.505970,89.679104,89.083582,74.040299,64.965672,56.753731,72.907090
1,1895,AR,45.998667,43.001333,62.096000,74.477333,79.480000,87.246667,87.884000,89.698667,87.817333,69.697333,59.329333,52.280000,69.917222
2,1895,AZ,51.220000,57.813333,66.253333,75.000000,81.713333,88.260000,91.186667,90.146667,85.060000,74.473333,58.413333,51.446667,72.582222
3,1895,CA,48.203448,56.387931,57.513793,64.218966,71.715517,81.587931,83.501724,84.015517,76.906897,71.694828,59.737931,50.400000,67.157040
4,1895,CO,33.342857,32.995238,45.574603,59.838095,65.365079,72.788889,76.246032,78.576190,74.719048,59.512698,41.812698,33.707937,56.206614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6188,2021,VT,27.250000,27.785714,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-78.663690
6189,2021,WA,41.753846,39.161538,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-76.507051
6190,2021,WI,27.975000,21.094444,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-79.160880
6191,2021,WV,39.465455,39.303636,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-99.900000,-76.685909


In [10]:
## Define parser to convert string date to year.
def parser(x):
    return datetime.strptime(x, '%Y')
series = pd.read_csv('county_max_temps.csv', header=0, index_col=14, parse_dates=True, squeeze=True, date_parser=parser)

In [11]:
## Convert index to series year.
series.index = series.index.to_period('Y')

In [20]:
## Create list of states to loop through
list_states = list(set(data['State']))


In [25]:
## initialize final table.
final = pd.DataFrame()

## Loop through all states.
for state in list_states:
    ## Reformat data.
    filtered = data[data['State'] == state]
    filtered = filtered[(filtered['Year'] < 2021) & (filtered['Year'] > 1970)]
    filtered['Year'] = pd.to_datetime(filtered['Year'], format='%Y')
    filtered.index = filtered['Year']
    filtered.index = filtered.index.to_period('Y')
    filtered = filtered['Avg_Temp']
    
    ## initialize max values for ARIMA parameters.
    p1 = 3
    d1 = 1
    q1 = 3
    
    ## Create list for AIC values for each model.
    list_aic = np.empty((p1,d1,q1))
    for p in range(p1):
        for d in range(d1):
            for q in range(q1):
                model = ARIMA(filtered, order=(p,d,q))
                model_fit = model.fit()

    list_aic1 = list_aic.flatten()
    index = np.where(list_aic1 == np.amin(list_aic1))[0][0]
    
    ## Find location with minimum AIC.
    opt_q = index % q1 
    opt_d = (index // q1) % d1 
    opt_p = index // (d1*q1)

    ## Create model with optimal parameters.
    optimal = ARIMA(filtered, order = (opt_p, opt_d, opt_q))
    optimal_fit = optimal.fit()
    
    ## Predict next 5 years and append to final table.
    output = filtered.copy()
    output = pd.DataFrame(output.append(optimal_fit.forecast(steps = 5))).rename(columns = {0:'Avg_Temp'})
    output['State'] = state
    final = final.append(output)
    

C:\Users\pjh00\miniconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


In [81]:
final = final.reset_index()

In [82]:
## Save file.
final = final.pivot(index = 'index', columns = 'State').reset_index()
final.to_csv('temp_proj.csv', index=False)